<a href="https://colab.research.google.com/github/PeteJoh/fpl-ai/blob/master/weekly_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# MAKE SURE TO ENTER CORRECT GW LOWER DOWN IN CODE
import requests
import pandas as pd

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [28]:



def get_team_name(id):
    general_information = requests.get(
        "https://fantasy.premierleague.com/api/bootstrap-static/"
    ).json()
    teams = general_information["teams"]
    for team in teams:
        if team["id"] == id:
            return team["name"]


def get_player_fixture_info(id):
    player_info = requests.get(
        f"https://fantasy.premierleague.com/api/element-summary/{id}/"
    ).json()
    fixture = player_info["fixtures"][0]
    home_team = get_team_name(fixture["team_h"])
    away_team = get_team_name(fixture["team_a"])
    kickoff_time = fixture["kickoff_time"]
    is_home = fixture["is_home"]
    return home_team, away_team, kickoff_time, is_home


def get_gameweek_result(id, gameweek):
    "returns a dictionary of the statistics from the gameweek"
    gameweek_results = requests.get(
        f"https://fantasy.premierleague.com/api/element-summary/{id}/"
    ).json()
    return gameweek_results["history"][gameweek - 1]


def get_player_position(element_type):
    general_information = requests.get(
        "https://fantasy.premierleague.com/api/bootstrap-static/"
    ).json()
    positions = general_information["element_types"]
    for position in positions:
        if position["id"] == element_type:
            return position["singular_name_short"]


general_info = requests.get(
    f"https://fantasy.premierleague.com/api/bootstrap-static/"
).json()
player_infos = general_info["elements"]
gameweek = 10
#***********************ENTER GW ABOVE THAT JUST COMPLETED***********************
"""gameweek_fixture = pd.read_csv(f"datasets/gw4.csv")
gameweek_fixture = pd.read_csv(
       f"https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2024-25/gws/gw4.csv"
    )

print(gameweek_fixture.columns)

def get_fixture(matchday, name):
    # load week's fixture
    global gameweek_fixture
    gameweek_fixture_ = gameweek_fixture[gameweek_fixture["name"] == name]
    #print(gameweek_fixture_["kickoff_time"].iloc[0])
    kickoff_time=gameweek_fixture_["kickoff_time"].iloc[0]
    team=gameweek_fixture_["team"].iloc[0]
    is_home=gameweek_fixture_["was_home"].iloc[0]
    opponent_team=get_team_name(gameweek_fixture_["opponent_team"].iloc[0])
    if is_home:
        home_team,away_team=team,opponent_team
    else:
        home_team,away_team=opponent_team,team

    return (
        home_team,
        away_team,
        kickoff_time,
        is_home
    )"""


gameweek_fixture = pd.read_csv(f"/content/gdrive/MyDrive/fplpredict/datasets/2024-25/fixtures/GW{gameweek}.csv")


def get_fixture(matchday, name):
    # load week's fixture
    global gameweek_fixture
    gameweek_fixture_ = gameweek_fixture[gameweek_fixture["name"] == name]
    return (
        gameweek_fixture_["home_team"].iloc[0],
        gameweek_fixture_["away_team"].iloc[0],
        gameweek_fixture_["kickoff_time"].iloc[0],
        gameweek_fixture_["is_home"].iloc[0],
    )

# week 6 results
all_players = []
for player_info in player_infos:
    try:
        #print(all_players)
        id = player_info["id"]
        print(id)
        name = player_info["first_name"] + " " + player_info["second_name"]
        team_id = player_info["team"]
        cost = player_info["now_cost"]
        element_type = player_info["element_type"]
        my_team = get_team_name(team_id)
        position = get_player_position(element_type)
        gameweek_result = get_gameweek_result(id, gameweek)
        home_team, away_team, kickoff_time, is_home = get_fixture(gameweek, name)
        #print(gameweek_result)
        gameweek_result["id"] = id
        gameweek_result["name"] = name
        gameweek_result["cost"] = cost
        gameweek_result["position"] = position
        gameweek_result["home_team"] = home_team
        gameweek_result["away_team"] = away_team
        gameweek_result["team_x"] = my_team
        gameweek_result["opponent_team"] = get_team_name(
            gameweek_result["opponent_team"]
        )
        all_players.append(gameweek_result)

    except IndexError:
        continue

# get x week result
print(all_players)
df = pd.DataFrame(all_players)
print(df)

1
2
3
4
5
6
7
8
9
12
13
15
16
17
18
20
21
22
23
24
25
73
186
578
597
633
638
639
657
658
659
662
665
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
233
569
598
640
641
656
59
60
61
62
63
64
65
66
67
68
69
70
71
72
74
75
76
77
78
79
80
81
83
84
85
152
580
592
599
617
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
314
338
602
634
649
661
113
114
115
116
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
595
600
601
615
618
628
629
663
668
150
151
153
154
155
157
158
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
187
189
386
560
575
581
621
11
159
190
193
194
195
196
197
199
200
201
202
203
205
206
207
208
209
210
211
212
213
214
215
216
445
585
644
645
646
647
650
666
156
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
234
235
236
237
238
577
582
60

In [ ]:
df

,element,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,minutes,...,selected,transfers_in,transfers_out,id,name,cost,position,home_team,away_team,team_x
0,1,84,Chelsea,0,False,2023-10-21T16:30:00Z,2,2,9,0,...,26039,0,1453,1,Folarin Balogun,44,FWD,Chelsea,Arsenal,Arsenal
1,2,84,Chelsea,0,False,2023-10-21T16:30:00Z,2,2,9,0,...,42211,2986,1814,2,Cédric Alves Soares,39,DEF,Chelsea,Arsenal,Arsenal
2,3,84,Chelsea,0,False,2023-10-21T16:30:00Z,2,2,9,0,...,8786,1006,682,3,Mohamed Elneny,44,MID,Chelsea,Arsenal,Arsenal
3,4,84,Chelsea,0,False,2023-10-21T16:30:00Z,2,2,9,0,...,12793,313,1175,4,Fábio Ferreira Vieira,54,MID,Chelsea,Arsenal,Arsenal
4,5,84,Chelsea,1,False,2023-10-21T16:30:00Z,2,2,9,90,...,1303504,106197,44609,5,Gabriel dos Santos Magalhães,47,DEF,Chelsea,Arsenal,Arsenal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
653,572,82,Bournemouth,0,False,2023-10-21T14:00:00Z,1,2,9,0,...,39501,2854,6654,572,Nélson Cabral Semedo,45,DEF,Bournemouth,Wolves,Wolves
654,573,82,Bournemouth,2,False,2023-10-21T14:00:00Z,1,2,9,74,...,8879,1147,377,573,Toti António Gomes,44,DEF,Bournemouth,Wolves,Wolves
655,574,82,Bournemouth,1,False,2023-10-21T14:00:00Z,1,2,9,90,...,47642,2132,3041,574,Boubacar Traoré,44,MID,Bournemouth,Wolves,Wolves
656,590,82,Bournemouth,7,False,2023-10-21T14:00:00Z,1,2,9,81,...,75828,15978,6998,590,Matheus Santos Carneiro Da Cunha,55,FWD,Bournemouth,Wolves,Wolves


In [31]:
df[df["team_x"]=="Man City"]

,element,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,minutes,...,selected,transfers_in,transfers_out,id,name,cost,position,home_team,away_team,team_x
374,340,80,Wolves,0,False,2024-10-20T13:00:00Z,1,2,8,0,...,526572,17712,25196,340,Manuel Akanji,55,DEF,Bournemouth,Man City,Man City
375,341,80,Wolves,0,False,2024-10-20T13:00:00Z,1,2,8,0,...,94422,131,5326,341,Nathan Aké,53,DEF,Bournemouth,Man City,Man City
376,342,80,Wolves,2,False,2024-10-20T13:00:00Z,1,2,8,90,...,552074,62088,37475,342,Bernardo Veiga de Carvalho e Silva,65,MID,Bournemouth,Man City,Man City
377,343,80,Wolves,0,False,2024-10-20T13:00:00Z,1,2,8,0,...,7473,124,553,343,Oscar Bobb,47,MID,Bournemouth,Man City,Man City
378,344,80,Wolves,0,False,2024-10-20T13:00:00Z,1,2,8,0,...,22648,972,887,344,Scott Carson,40,GKP,Bournemouth,Man City,Man City
379,345,80,Wolves,0,False,2024-10-20T13:00:00Z,1,2,8,0,...,432856,1412,45037,345,Kevin De Bruyne,94,MID,Bournemouth,Man City,Man City
380,346,80,Wolves,5,False,2024-10-20T13:00:00Z,1,2,8,65,...,240710,24026,16027,346,Jérémy Doku,63,MID,Bournemouth,Man City,Man City
381,347,80,Wolves,2,False,2024-10-20T13:00:00Z,1,2,8,90,...,916111,68738,45519,347,Ederson Santana de Moraes,55,GKP,Bournemouth,Man City,Man City
382,348,80,Wolves,4,False,2024-10-20T13:00:00Z,1,2,8,24,...,689204,143073,35954,348,Phil Foden,93,MID,Bournemouth,Man City,Man City
383,349,80,Wolves,1,False,2024-10-20T13:00:00Z,1,2,8,17,...,38211,6818,3831,349,Jack Grealish,64,MID,Bournemouth,Man City,Man City


In [29]:
df[df["team_x"]=="Liverpool"]

,element,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,minutes,...,selected,transfers_in,transfers_out,id,name,cost,position,home_team,away_team,team_x
346,310,74,Chelsea,0,True,2024-10-20T15:30:00Z,2,1,8,0,...,763971,2438,231099,310,Alisson Ramses Becker,54,GKP,Liverpool,Brighton,Liverpool
347,311,74,Chelsea,2,True,2024-10-20T15:30:00Z,2,1,8,80,...,3228663,82832,141559,311,Trent Alexander-Arnold,71,DEF,Liverpool,Brighton,Liverpool
348,312,74,Chelsea,0,True,2024-10-20T15:30:00Z,2,1,8,0,...,5759,0,511,312,Stefan Bajčetić Maquieira,45,MID,Liverpool,Brighton,Liverpool
349,313,74,Chelsea,0,True,2024-10-20T15:30:00Z,2,1,8,0,...,33361,424,2598,313,Conor Bradley,47,DEF,Liverpool,Brighton,Liverpool
350,315,74,Chelsea,0,True,2024-10-20T15:30:00Z,2,1,8,0,...,2037,0,124,315,Bobby Clark,45,MID,Liverpool,Brighton,Liverpool
351,316,74,Chelsea,1,True,2024-10-20T15:30:00Z,2,1,8,60,...,225143,5671,23273,316,Darwin Núñez Ribeiro,71,FWD,Liverpool,Brighton,Liverpool
352,317,74,Chelsea,1,True,2024-10-20T15:30:00Z,2,1,8,29,...,883765,71377,75761,317,Diogo Teixeira da Silva,73,MID,Liverpool,Brighton,Liverpool
353,318,74,Chelsea,0,True,2024-10-20T15:30:00Z,2,1,8,0,...,2450,0,168,318,Ben Doak,45,MID,Liverpool,Brighton,Liverpool
354,319,74,Chelsea,0,True,2024-10-20T15:30:00Z,2,1,8,0,...,26853,74,1903,319,Harvey Elliott,52,MID,Liverpool,Brighton,Liverpool
355,320,74,Chelsea,0,True,2024-10-20T15:30:00Z,2,1,8,0,...,16230,697,1253,320,Endo Wataru,47,MID,Liverpool,Brighton,Liverpool


In [30]:
#save to folder
df.to_csv(f"/content/gdrive/MyDrive/fplpredict/datasets/2024-25/results/GW{gameweek}.csv")